In [1]:
import numpy as np
import scipy
import time
import math
from scipy.stats import norm

import mystic
from mystic.solvers import DifferentialEvolutionSolver, diffev2
from mystic.strategy import Best1Bin
from mystic.monitors import Monitor,VerboseMonitor

from copy import deepcopy

from Tomography import *

from NestedForLoop import get_iterator
from pathlib import Path
from scipy.linalg import sqrtm

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import os
import glob

import pandas as pd

from scipy.optimize import least_squares

import fnmatch
from Efficiencies import finding_file, get_channels_eff, set_raw_counts
from Optimization import Optimizer, function_fidelity, FidelityResults
from constants import *

from DensityMatrix import DensityMatrix, apply_unitary_to_dm

from pathlib import Path
import fnmatch

In [2]:
######################################################################################################
#----- COUNTING THE FILES AND SAVING THEM IN AN ARRAY TO MAKES THE REST OF THE ANALYSIS EASIER -------
######################################################################################################

n_files=0
working_dir=r"G:\Other computers\Mon ordinateur\ChannelCertification\SteeringHonest"
os.chdir(working_dir)

filenames = [i for i in glob.glob("Channel*")]
filenames.sort(key=os.path.getmtime)

index_to_file = {}

for index, filename in enumerate(filenames):
    os.chdir(f"{working_dir}\\{filename}")
    filenames_aux=[i for i in glob.glob("StateTomography*")]
    for index_second, filenames_aux_second in enumerate(filenames_aux):
        index_to_file[n_files] = f"{filename}\\{filenames_aux_second}"
        n_files+=1
os.chdir(working_dir)

In [3]:
os.chdir(working_dir)
qubit_number=2

state_after=[]
xp_counts_after_corrected_with_eff=[]
state_after_file=[]

state_before=[]
xp_counts_before_corrected_with_eff=[]
state_before_file=[]
state = []
xp_counts_corrected_with_eff=[]

#####################################################################
#---------------------- STATE TOMOGRAPHY ----------------------------
#####################################################################

for index in range(len(index_to_file)):
    os.chdir(f"{working_dir}\\{index_to_file[index]}\\StateTomo")
    datafiles=[i for i in glob.glob("*")]
                
    ### Calculating the efficiencies of each detector
    efficiencies=get_channels_eff(datafiles, os.getcwd())

    ### Opening the data files and writing the data in counts_aux array
    counts_aux=set_raw_counts(datafiles, qubit_number, os.getcwd())
    xp_counts=np.array(np.transpose(counts_aux))

    statetomo=LRETomography(int(qubit_number), xp_counts)
    statetomo.run(correct_eff=efficiencies, print_nc=False)
    xp_counts_corrected_with_eff.append(statetomo.xp_counts)

    ## The 'e' and 'r' serve to distinguish between tomography before and after, respectively
    ## We want to save them in different arrays because we need them for different things
    if index_to_file[index][-8]=='e':
        state_before.append(DensityMatrix(statetomo.quantum_state.get_density_matrix()))
        xp_counts_before_corrected_with_eff.append(statetomo.xp_counts)
        state_before_file.append(index_to_file[index])
        #print('\n Fast maximum likelihood estimation: \n', state_before[-1], '\n')

    elif index_to_file[index][-8]=='r':
        state_after.append(DensityMatrix(statetomo.quantum_state.get_density_matrix()))
        xp_counts_after_corrected_with_eff.append(statetomo.xp_counts)
        state_after_file.append(index_to_file[index])
        #print('\n Fast maximum likelihood estimation: \n', state_after[-1], '\n')
            

In [4]:
######################################
#-- DEFINING THE TARGET BELL STATE ---
######################################

#bell=(np.array([1,0,0,0])+np.array([0,0,0,1]))/np.sqrt(2)
#bell=(np.array([1,0,0,0])-np.array([0,0,0,1]))/np.sqrt(2)
bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
#bell=(np.array([0,1,0,0])-np.array([0,0,1,0]))/np.sqrt(2)
bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))

In [5]:
# states=state_after
# for index in range(len(states)):
#     print(np.real(np.round(states[index].fidelity_to_pure(bell),5)))

In [6]:
##########################################################
#----- OPTIMIZATION OF MAX FIDELITY UP TO UNITARIES ------
##########################################################
bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
fid=np.zeros((n_files))
optimized_matrix=np.zeros((n_files,2**qubit_number,2**qubit_number), dtype='complex')

guess=np.array([0, 0, 0, 0, 0, 0])
bounds=[(-np.pi,np.pi)]*6
results = []

opt=Optimizer(guess, function_fidelity, results=FidelityResults)

for index in range(len(state_before)):
    result=opt.optimize(state_before[index], bell, bounds=bounds)
    results.append(result)


Generation 0 has ChiSquare: -0.080570
Generation 50 has ChiSquare: -0.994337
Generation 100 has ChiSquare: -0.994337
Generation 150 has ChiSquare: -0.994337
STOP("ChangeOverGeneration with {'tolerance': 1e-08, 'generations': 100}")
Optimization terminated successfully.
         Current function value: -0.994337
         Iterations: 183
         Function evaluations: 18400
Generation 0 has ChiSquare: -0.033834
Generation 50 has ChiSquare: -0.991914
Generation 100 has ChiSquare: -0.991920
Generation 150 has ChiSquare: -0.991920
Generation 200 has ChiSquare: -0.991920
STOP("ChangeOverGeneration with {'tolerance': 1e-08, 'generations': 100}")
Optimization terminated successfully.
         Current function value: -0.991920
         Iterations: 237
         Function evaluations: 23800
Generation 0 has ChiSquare: -0.011141
Generation 50 has ChiSquare: -0.990963
Generation 100 has ChiSquare: -0.990965
Generation 150 has ChiSquare: -0.990965
Generation 200 has ChiSquare: -0.990965
STOP("ChangeO

In [42]:
##########################################################
#---------------- CALCULATING THE ERRORS -----------------
##########################################################
bell=(np.array([0,1,0,0])+np.array([0,0,1,0]))/np.sqrt(2)
error_runs=1000

mu=np.zeros((n_files))
std=np.zeros((n_files))

U=[]
bell_aux=[]
target=[]
states_final=[]

states=state_before #[DensityMatrix(x.optimized_state) for x in results] #
xp_counts_corrected_with_eff=xp_counts_before_corrected_with_eff

for index in range(len(states)):
    
    #bell_aux is the state after applying the inverse of the unitaries found in the optimization
    U.append(np.kron(results[index].u1,results[index].u2))
    target.append(np.linalg.inv(U[-1])@state_after[index].state@np.linalg.inv(np.transpose(np.conjugate(U[-1]))))
    #bell_aux.append(np.conjugate(np.transpose(U))@bell)
    
    dm = states[index]#.optimizedapply_unitary(U[-1])
    dm.calculate_errors(xp_counts_corrected_with_eff[index].counts_array, error_runs, target[-1])
    states_final.append(dm)
    
    print('index, fidelity, fidelity_mean, fidelity_std: ',
          index, np.round(states_final[-1].fidelity(target[-1]),5), np.round(states_final[-1].mu,5),
          np.round(states_final[-1].std,5))

index, fidelity, fidelity_mean, fidelity_std:  0 (0.99274+0j) 0.85489 0.00085
index, fidelity, fidelity_mean, fidelity_std:  1 (0.99921-0j) 0.95899 0.00059
index, fidelity, fidelity_mean, fidelity_std:  2 (0.01348-0j) 0.0141 0.00041
index, fidelity, fidelity_mean, fidelity_std:  3 (0.99464-0j) 0.98491 0.00029
index, fidelity, fidelity_mean, fidelity_std:  4 (0.99478+0j) 0.98223 0.00036
index, fidelity, fidelity_mean, fidelity_std:  5 (0.98964-0j) 0.96659 0.00053
index, fidelity, fidelity_mean, fidelity_std:  6 (0.99193+0j) 0.96272 0.00052
index, fidelity, fidelity_mean, fidelity_std:  7 (0.99621+0j) 0.90784 0.00051
index, fidelity, fidelity_mean, fidelity_std:  8 (0.9964+0j) 0.9 0.00041
index, fidelity, fidelity_mean, fidelity_std:  9 (0.99564+0j) 0.82788 0.00063
index, fidelity, fidelity_mean, fidelity_std:  10 (0.99365+0j) 0.97794 0.00054
index, fidelity, fidelity_mean, fidelity_std:  11 (0.9862-0j) 0.94602 0.00066


In [44]:
bell=(np.array([0,1,0,0])-np.array([0,0,1,0]))/np.sqrt(2)
bellmatrix=np.array(np.outer(bell, np.conjugate(bell)))

result=opt.optimize(state_before[2], bell, bounds=bounds)
U[2]=np.kron(result.u1,result.u2)
target[2]=np.linalg.inv(U[2])@state_after[2].state@np.linalg.inv(np.transpose(np.conjugate(U[2])))

dm = states[2]
dm.calculate_errors(xp_counts_corrected_with_eff[2].counts_array, error_runs, target[2])
states_final[2]=dm

print('index, fidelity, fidelity_mean, fidelity_std: ',
          index, np.round(states_final[2].fidelity(target[2]),5), np.round(states_final[2].mu,5),
          np.round(states_final[2].std,5))

Generation 0 has ChiSquare: -0.003392
Generation 50 has ChiSquare: -0.990965
Generation 100 has ChiSquare: -0.990965
Generation 150 has ChiSquare: -0.990965
STOP("ChangeOverGeneration with {'tolerance': 1e-08, 'generations': 100}")
Optimization terminated successfully.
         Current function value: -0.990965
         Iterations: 180
         Function evaluations: 18100
index, fidelity, fidelity_mean, fidelity_std:  11 (0.99632+0j) 0.99364 0.00081


In [45]:
##########################################################
#------------ WRITING THE DATA IN AN EXCEL ---------------
##########################################################

import xlsxwriter
states=state_before

workbook = xlsxwriter.Workbook('fidelities_to_ini_state_errors.xlsx') ### We should write this in another place
 
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'Number')
worksheet.write('B1', 'Folder')
worksheet.write('C1', 'Fidelity')
worksheet.write('D1', 'Fidelity_mean')
worksheet.write('E1', 'Fidelity_uncertainty')

counter=0
for index in range(len(state_before)):
    
    worksheet.write('A'+str(counter+2), counter)
    worksheet.write('B'+str(counter+2), state_before_file[index])
    worksheet.write('C'+str(counter+2), np.real(np.round(states_final[index].fidelity(target[index]),5)))#np.round(states[index].fidelity_to_pure(bell),5)))
    worksheet.write('D'+str(counter+2), np.real(np.round(states_final[index].mu,5)))
    worksheet.write('E'+str(counter+2), np.real(np.round(states_final[index].std,5)))
    counter+=1
        
workbook.close()

In [ ]:
### IF WE WANT TO CHECK THE FIT TO THE UNCERTARTAINTY
### Then we need to calculate the statistic on these simulated fidelities and calculate the standart deviation
### This will be our uncertainty due to statistical errors
def count_elements(seq) -> dict:
    hist = {}
    for i in seq:
        hist[i] = hist.get(i, 0) + 1
    return hist

counted = count_elements(fidelity_sim)
#print(counted)
bin_numb = len(counted)
errorbar_x=np.array(list(counted))
errorbar_y=np.zeros((bin_numb), dtype=int)

for i in range(bin_numb):
    errorbar_y[i]=counted[errorbar_x[i]]
    
#print(errorbar_x)
#print(errorbar_y)
    
def Gauss(x, A, mu, sigm):
    y = A*np.exp(-((x-mu)/sigm)**2/2)
    return y

mu, std = norm.fit(fidelity_sim)
print(mu, std)
parameters, covariance = curve_fit(Gauss, xdata=errorbar_x[-1], ydata=errorbar_y[:-1], bounds=[(0,0.99007,1e-4),(60,0.9903,0.0002)])
fit_A = parameters[0]
fit_B = parameters[1]
fit_C = parameters[2]
print(fit_A, fit_B, fit_C)

xdata= x = np.linspace(0.989, 0.991, 100)
fit_y = Gauss(xdata, fit_A, fit_B, fit_C)
plt.plot(errorbar_x[:-1], errorbar_y[:-1], 'o', label='data')
plt.plot(xdata, fit_y, '-', label='fit')
plt.legend()